Sequence labeling on Twitter data using Generative Classifiers
=====================

This project focuses on sequence labeling. 
Part (a) focuses on *generative* approaches, including Naive Bayes and the Hidden Markov Model (HMM).
The target domain is Twitter part-of-speech tagging.

In [1]:
import scorer, operator
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
%pylab --no-import-all inline

Populating the interactive namespace from numpy and matplotlib


# 1. Data Processing #

Download the train data and dev data from [Github](https://github.com/jacobeisenstein/gt-nlp-class/tree/master/projects/proj-2). 
The test data will be released around 48 hours before the deadline for Pset 2b.
The part-of-speech tags are defined in the [ACL2011 paper](http://www.ark.cs.cmu.edu/TweetNLP/gimpel+etal.acl11.pdf) 
and the [NAACL 2013 paper](http://www.ark.cs.cmu.edu/TweetNLP/owoputi+etal.naacl13.pdf), 
which also describe the data and gives some state-of-art results.

In [2]:
"""
Data processing code
"""
def conllSeqGenerator(input_file,max_insts=1000000):
    """ return an instance generator for a filename
    
    The generator yields lists of words and tags.  
    """
    cur_words = []
    cur_tags = []
    num_insts = 0
    with open(input_file) as instances:
        for line in instances:
            if len(line.rstrip()) == 0:
                if len(cur_words) > 0:
                    num_insts += 1
                    yield cur_words,cur_tags
                    cur_words = []
                    cur_tags = []
            else:
                parts = line.rstrip().split()
                cur_words.append(parts[0])
                if len(parts)>1:
                    cur_tags.append(parts[1])
                else: cur_tags.append(unk)
        if len(cur_words)>0: 
            num_insts += 1
            yield cur_words,cur_tags

In [3]:
## Define the file names
trainfile = 'oct27.train'
devfile = 'oct27.dev'
testfile = 'oct27.test' # You do not have this for now
offset = "**OFFSET**"
unknown = "**UNKNOWN**"

Here is a demo code for using function "conllSeqGenerator()"

In [4]:
## Demo
alltags = set()
distinct_words = set()
word_counters = Counter()
for i,(words, tags) in enumerate(conllSeqGenerator(trainfile)):    
    for tag in tags:
        alltags.add(tag)
    for word in words:
        distinct_words.add(word)
        word_counters += Counter(words)
print alltags


set(['!', '#', '$', '&', ',', 'A', '@', 'E', 'D', 'G', 'M', 'L', 'O', 'N', 'P', 'S', 'R', 'U', 'T', 'V', 'Y', 'X', 'Z', '^', '~'])


**Deliverable 1a** (1 point): Use the Counter class to identify the most common three words for each tag, in the training set. 
The most_common() function of the Counter class will help you here. 

In [5]:

import itertools
dict_tag = defaultdict(list)
for i, (words, tags) in enumerate(conllSeqGenerator(trainfile)):
    for word,tag in itertools.izip(words,tags):
        dict_tag[tag].append(word)

dict_mostCommon = defaultdict(list)
for v,u in dict_tag.iteritems():
    t_list= Counter(u).most_common(3)   
    dict_mostCommon[v].append(t_list)

print len(dict_mostCommon.keys())
for v , u in dict_mostCommon.items():
    print v,u


25
! [[('lol', 54), ('Lol', 22), ('oh', 10)]]
# [[('#nowplaying', 3), ('#np', 2), ('#tcot', 2)]]
$ [[('one', 32), ('4', 6), ('2010', 6)]]
& [[('and', 117), ('&', 39), ('but', 31)]]
, [[('.', 427), ('!', 244), (',', 225)]]
A [[('good', 24), ('new', 22), ('more', 13)]]
@ [[('@Fresh32Prince89', 6), ('@lil_jeezy_85', 2), ('@ResourcefulMom', 2)]]
E [[(':)', 28), ('<3', 10), (';)', 8)]]
D [[('the', 236), ('a', 165), ('my', 89)]]
G [[('smh', 9), ('|', 7), ('-', 7)]]
M [[("momma's", 1), ('#LebronShould', 1), ("Ricochet's", 1)]]
L [[("I'm", 42), ('its', 24), ('im', 15)]]
O [[('I', 258), ('you', 135), ('it', 87)]]
N [[('day', 19), ('time', 18), ('people', 17)]]
P [[('to', 231), ('of', 112), ('for', 101)]]
S [[('mans', 1), ("judge's", 1), ('year\xe2\x80\x99s', 1)]]
R [[('just', 56), ('not', 27), ('now', 26)]]
U [[('http', 4), ('http://tumblr.com/xeinfgrpl', 1), (':/', 1)]]
T [[('out', 29), ('up', 26), ('on', 8)]]
V [[('is', 105), ('are', 52), ('have', 48)]]
Y [[("there's", 2)]]
X [[('all', 6), ('

## Most-common-tag baseline ##

Build a classifier that chooses the most common tag for each word.

- You should implement your classifier in terms of a set of weights, as in pset 1
- Prediction should use your predict() function from pset 1
- For unseen words, the classifier should choose the tag with the most **unique** word types

In [6]:
argmax = lambda x : max(x.iteritems(),key=operator.itemgetter(1))[0]

In [7]:
# paste your predict function here
# should return two outputs: the highest-scoring label, and the scores for all labels
def predict(instance,weights,labels):
    # scores
    scores = defaultdict(int)
    for w , count in instance.items():
        for label in labels:
            scores[label] = scores.get(label,0) + count*weights.get((label,w),0)
    return argmax(scores),scores

evalTagger evaluates a tagger. It takes three arguments:

- a tagger, which is a **function** taking a list of words and a tagset as arguments
- an output filename
- a test file

You will want to use lambda expressions to create the first argument for this function, as shown below.

In [8]:
def evalTagger(tagger,outfilename,testfile=devfile):    
    with open(outfilename,'w') as outfile:
        for words,_ in conllSeqGenerator(testfile):
            pred_tags = tagger(words,alltags)
            for tag in pred_tags:
                print >>outfile, tag
            print >>outfile, ""
    return scorer.getConfusion(testfile,outfilename) #run the scorer on the prediction file

Here's how it works. I provide a tagger that labels everything as a noun.

In [9]:
# the list comprehension creates a list of 'N' equal to the length of the list of words
noun_tagger = lambda words, alltags : ['N' for word in words]
confusion = evalTagger(noun_tagger,'nouns')
print scorer.accuracy(confusion)

0.136844287788


Define a function called classifier tagger, with the following signature:
- **input 1** a list of words
- **input 2** a dict of weights
- **input 3** a list of all possible tags
- **output 1** a list of predicted tags

Your function should call predict, using the weights.

In [10]:
# define a function called classifierTagger
def classifierTagger(words,weights,all_tags):
    tags = []
    scores = defaultdict(float)
    # your code here
    for word in words:
        tag,scores = predict({word:1,offset:1}, weights, all_tags)   # since a single instance consists of a single word
        tags.append(tag)
    return tags

**Deliverable 1b** (1 point) Use your classifierTagger to reproduce the "always noun" tagger above, this time by using the offset weight. 

**Sanity check** the accuracy should be the same as above

In [11]:
your_weights = defaultdict(float)
for tag in alltags:
    for word in words:
        if tag == 'N':
            your_weights.update({(tag,word) : 0.0})
            your_weights.update({(tag,offset) : 1.0})
        else:
            your_weights.update({(tag,word) : 0.0})
            your_weights.update({(tag,offset): 0.0})

noun_tagger_2 = lambda words, alltags: classifierTagger(words, your_weights, alltags)
print scorer.accuracy(evalTagger(noun_tagger_2,'nouns'))

0.136844287788


**Deliverable 1c** (2 points) Implement a tagger that selects the most common tag for each word. 
- You should use classifierTagger for this, so select the weights appropriately
- To do this, you'll want to construct a bunch of counters, similar to what you did above.
- For words that do not appear in the training data, select the tag which applies to the most word *types* in the training data.

In [12]:
def weightsInitialize():
    weights = defaultdict(float)
    for tag in alltags:
        for word in words:
            weights.update({(tag,word) : 0.0})
            weights.update({(tag,offset) : 0.0})
    return weights

weights = weightsInitialize()

tag_dict = defaultdict(list)
for i, (words, tags) in enumerate(conllSeqGenerator(trainfile)):
    for word,tag in itertools.izip(words,tags):
        tag_dict[word].append(tag)

alltag_Counter = Counter();
for v,u in tag_dict.iteritems():
    most_occur_labeltag =  Counter(u).most_common(1)
    alltag_Counter.update(most_occur_labeltag[0][0])
    weights.update({(most_occur_labeltag[0][0],v): 1})
        
mostCommontag = alltag_Counter.most_common(1) 
for y in t:
    weights.update({(mostCommontag[0][0], offset) : 0.8})


NameError: name 't' is not defined

**Sanity check** my accuracy is approximately 70%

In [27]:
confusion = evalTagger(lambda words,alltags : classifierTagger(words,weights,alltags),'mcc')
print scorer.accuracy(confusion)

0.702260004147


# 2 Naive Bayes Classification #


Write a function that builds a Naive Bayes classifier to perform supervised classification.

- The base features should just be the word itself, plus an offset feature. 
- The output of this function should be weights that you can plug into the predict function that you wrote last time
- An input should be a smoothing value $\alpha$
- **Hint**: You may want to use the counters that you built in problem 1a.

In [37]:
Vset = set()
total_words = 0 
V = 0

for i, (words, tags) in enumerate(conllSeqGenerator(trainfile)):
    total_words += len(words)
    for word in words:
        Vset.add(word)
V = len(Vset)
        
# print total_words , V

def getNBWeights(alpha):
    sum_dict = defaultdict(float) 
    for tag,wordlist in d.iteritems():
        c = Counter(wordlist)
        sum_c = sum(c.values())
        sum_dict.update({tag : sum_c})

    num = float()
    den = float()
    log_phi = float()
    bayes_weights = defaultdict(lambda : -1000.) #default value is a low log-probability
    
    for word in distinct_words:
        for tag,val in sum_dict.iteritems():
            bayes_weights.update({(tag,word): (np.log(alpha)- np.log(val + V*alpha))} )  #initialed to a default smoothed value
                                  
    for tag,wordlist in dict_tag.iteritems():
        c = Counter(wordlist)
        sum_c = sum(c.values())
        sum_dict.update({tag : sum_c})
        type_c = len(c.keys())

        for word,count  in c.iteritems():
            num = count + alpha
            den = sum_c + V * alpha
            log_phi = np.log(num) - np.log(den)
            bayes_weights.update({(tag,word): log_phi})
        bayes_weights.update({(tag,offset) : (np.log(sum_c)- np.log(total_words))})

    return bayes_weights

**Sanity check**: your code should give the same results I get below

In [38]:
w1 = getNBWeights(0.1)
# print total probability mass for a tag
print sum([np.exp(w1[('N',word)]) for word in word_counters.keys()])
# print some common values
print w1[('N','breakfast')], w1[('V','breakfast')], w1[('A','smart')], w1[('D','the')], w1[('!',offset)]


1.0
-7.74708642426 -10.226404038 -6.42687365064 -1.78871941598 -3.58372417191


**Deliverable 2** (3 points): run the code below to evaluate your naive bayes tagger on the development set.

**Sanity check**: you may do a little worse than the most-common tag classifier

In [40]:
dev_acc = dict()
for alpha in [1e-4,1e-3,1e-2,1e-1,1e0,1e1]:
    nb_weights = getNBWeights(alpha)
    confusion = evalTagger(lambda words, alltags : classifierTagger(words,nb_weights,alltags),'nb')
    dev_acc[alpha] = scorer.accuracy(confusion)
    print alpha,dev_acc[alpha]

0.0001 0.67095168982
0.001 0.67095168982
0.01 0.67095168982
0.1 0.66846361186
1.0 0.625544267054
10.0 0.510470661414


# 3 Viterbi Algorithm #

In this section you will implement the Viterbi algorithm. As a reminder, here's the math:

$\renewcommand{\vec}[1]{\mathbf{#1}}$
\begin{align*}
\vec{w}^{\top}\vec{f}(\vec{x},\vec{y}) = & \sum_i \vec{w}^{\top} \vec{f}(\vec{x},y_i,y_{i-1},i) \\
v(y,0) = & \vec{w}^{\top}\vec{f}(\vec{x},y,\diamond,0)\\
b(y,0) = & \diamond \\
v(y,i) = & \max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
b(y,i-1) = & \text{arg}\max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
\end{align*}

To get warmed up, let's work out an example by hand. There are only two tags, 
N and V. Here are the parameters:

| | Value |
| ------------- |:-------------:|
| $\log P_E(\cdot|N)$ | they: -1, can: -3, fish: -3 |
| $\log P_E(\cdot|V)$ | they: -10, can: -2, fish: -3 |
| $\log P_T(\cdot|N)$ | N: -5, V: -2, END: -2 |
| $\log P_T(\cdot|V)$ | N: -1, V: -4, END: -3 |
| $\log P_T(\cdot|\text{START})$ | N :-1, V :-1 |

where $P_E(\cdot|\cdot)$ is the emission probability and $P_T(\cdot|\cdot)$ is the translation probability.
 
In class we discuss the sentence *They can fish*. Now work out a more complicated example: "*They can can fish*".
 
** Deliverable 3a ** (1 point) Show the trellis-like table, and give the score for the best best scoring path(s). After you work out the trellis by hand, you should be able to fill the following table.


** Sanity check ** There are two paths that each score -18.

*(Fill your answer in the following table)*

|POS tag| START  | they | can | can | fish | END |
|-------|:-------|:-----|:----|:----|:-----|:---:|
| N     |    0   | -2   | -10 | -10 | -16  | -18 |
| V     |    0   | -11  | -6  | -12 | -15  | -18 |

## Implementing Viterbi ##

In [41]:
# you can see how these are used in the example weights below
start_tag = '--START--'
end_tag = '--END--'
trans ='--TRANS--'
emit = '--EMISSION--'

The following function computes the HMM features for the function $\vec{f}(\vec{x},y,y',i)$. 
- You will call it in your viterbi tagger. 
- Note that it returns both an emission and transition feature, except for the last word, where it returns only a transition feature. 
- Also note that transition and emission features are specially marked

In [42]:
def hmm_feats(words,curr_tag,prev_tag,i):
    if i < len(words):
        return [(curr_tag,words[i],emit),(curr_tag,prev_tag,trans)]
    else: return [(curr_tag,prev_tag,trans)]

Here are some predefined weights, corresponding to problem 3a.

In [43]:
defined_weights = {('N','they',emit):-1,('N','can',emit):-3,('N','fish',emit):-3,\
                        ('V','they',emit):-10,('V','can',emit):-2,('V','fish',emit):-3,\
                        ('N','N',trans):-5,('V','N',trans):-2,(end_tag,'N',trans):-3,\
                        ('N','V',trans):-1,('V','V',trans):-4,(end_tag,'V',trans):-3,\
                        ('N',start_tag,trans):-1,('V',start_tag,trans):-1}

Define viterbiTagger

- **Input 1**: a list of words
- **Input 2**: a feature function, like hmm_feats
- **Input 3**: a dict of weights
- **Input 4**: a list of all possible tags
- **Output 1**: the best-scoring sequence
- **Output 2**: the score of the best-scoring sequence

Hint: you can represent the trellis and the back pointers as lists of dicts. You will want to do some special handling for the first and last words; otherwise, just iterate 

In [66]:
def viterbiTagger(words,feat_func,weights,all_tags,debug=False):
    trellis = [None] * len(words) #hint: store the $v$ table here 
    pointers = [None] * len(words) #hint: store the $b$ table here
    output = [None] * len(words) #hint: store the output here. build this last.

    all_tags = list(all_tags)
    
#   Matrix represents the trellis structure for m words and t classes. Represented as lists of list
    Matrix = [[100 for x in xrange(len(words)+1)] for x in xrange(len(all_tags))] 
   
#     The b matrix is a pointer matrix. Represented as lists of list
    b = [[100 for x in xrange(len(words)+1)] for x in xrange(len(all_tags))]   
    

    for i in xrange(len(words)+1):
        for j in xrange(len(all_tags)):
            temp_weight = list()
            if (i-1) < 0:                                                            
                temp = feat_func(words,all_tags[j],start_tag,i)          
                Matrix[j][i] = weights [temp[0]] + weights[temp[1]]
                b[j][i] = -1

            else:
                for k in xrange(len(all_tags)):
                    if( i == len(words)):
                        temp = feat_func(words,end_tag,all_tags[k],len(words))
                        temp_weight.append(weights[temp[0]] + Matrix[k][i-1])
                    else:
                        temp = feat_func(words,all_tags[j],all_tags[k],i)
                        temp_weight.append(weights[temp[1]] + Matrix[k][i-1])
                maxi = temp_weight[0]
                pos = 0
                for z in xrange(len(temp_weight)):
                    if temp_weight[z] > maxi:
                        maxi = temp_weight[z]
                        pos = z
                b[j][i] = pos
                if( i == len(words)):
                     Matrix[j][i] = maxi
                else:
                    Matrix[j][i] = weights[temp[0]] + maxi

    m = Matrix[0][len(words)]
    pos = 0
    for z in xrange(len(temp_weight)):
        if (temp_weight[z] > m):
            m = temp_weight[z][len(words)]
            pos = z
    best_score = Matrix[0][len(words)] 

    index = pos
    cntr = len(words)
    while(cntr > 0):
        index = b[index][cntr]
        output[cntr-1] = all_tags[index]
        cntr -= 1

    return output,best_score  



**Deliverable 3b** (1 point) run you viterbi tagger on the example in 3a, using the code below.

In [67]:
# keep this code
viterbiTagger(['they','can','can','fish'],hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V'], -18)

**Deliverable 3c** (1 point) run your Viterbi on the following example

In [68]:
sent = 'they can can can can can can can fish'.split()

In [69]:
viterbiTagger(sent,hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V', 'N', 'V', 'N', 'V', 'N'], -37)

Now estimate the weights of a hidden Markov model. 
- You have already estimated the emission weights $\log P(w | y)$, in your solution to problem 2. Use your solution with $\alpha=0.001$
- Estimate the transition probabilities from the training data, using the maximum-likelihood estimates (no smoothing)
- Don't forget transitions from the start state and to the end state

In [70]:
nb_weights = getNBWeights(0.001) #compute naive bayes weights
# convert nb weights to hmm weights
hmm_weights = defaultdict(lambda : -1000.)
# your code here
for (tag,word),val in nb_weights.iteritems():
    hmm_weights.update({(tag,word,emit): val})


In [71]:
# compute tag-to-tag transition counts, add them to hmm_weights
tag_trans_counts = defaultdict(lambda : Counter())
new_taglist = list()
import itertools

sum_dict = defaultdict(float) 
for tag,wordlist in d.iteritems():
    c = Counter(wordlist)
    sum_c = sum(c.values())
    sum_dict.update({tag : sum_c})
        
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return itertools.izip(a, b)

for i, (words, tags) in enumerate(conllSeqGenerator(trainfile)):
    tags_list = [start_tag] + tags + [end_tag]
    for fcntr, scntr in pairwise(tags_list):
            tag_trans_counts[(scntr,fcntr,trans)]   =   tag_trans_counts.get((scntr,fcntr,trans),0)+1

# updating the counts of start and end tag which have been inserted at start and end of each sentence respectively
sum_dict.update({start_tag : i})      
sum_dict.update({end_tag : i})

for u,v in tag_trans_counts.iteritems():
    hmm_weights.update({u : (np.log(v) - np.log(sum_dict[u[1]]))})
    


**Sanity check**: here are some weights that I estimate. Yours should be very close, if not identical.

In [72]:
print hmm_weights['V','go',emit], hmm_weights['~','go',emit], hmm_weights['^','diddy',emit]
print hmm_weights['V','V',trans], hmm_weights['~','V',trans]
print hmm_weights[end_tag,'V',trans], hmm_weights[end_tag,'~',trans]

-4.66268727503 -13.2056617029 -1000.0
-1.89367092996 -5.9130524537
-4.30361454127 -3.06898273529


**Sanity check**: here's the tag sequence and score for a modified version of our example sentence

In [73]:
viterbiTagger([':))','we','can','can','fish',':-)'],hmm_feats,hmm_weights,alltags)

(['E', 'O', 'V', 'V', 'N', 'E'], -47.969198217657684)

**Deliverable 3d** (3 points): compute the predicted tag sequence and scores for the first three sentence in the training data.

In [74]:
for i,(words,_) in enumerate(conllSeqGenerator(trainfile)):
    print i, viterbiTagger(words,hmm_feats,hmm_weights,alltags)
    if i >= 2: break

0 (['O', 'V', 'O', 'V', 'V', 'D', 'A', 'N', 'O', 'V', 'T', ',', 'V', '^', '^', 'N', ',', 'R', 'P', 'O', 'V', 'L', 'P', 'O', '~', '@', '~', '^', ',', '~', ',', 'U'], -204.4159428164603)
1 (['~', '@', '~', 'O', 'N', 'V', 'P', 'D', 'N', 'N', ','], -77.941023943930929)
2 (['^', 'A', '^', '$', ',', 'V', 'D', 'A', 'N', 'E'], -71.352279694472855)


** Deliverable 3d** (2 points):
- Run your HMM tagger on the dev data, using the code line below.
- ** Sanity check**: I get 74.5% accuracy

In [75]:
confusion = evalTagger(lambda words, alltags : viterbiTagger(words,hmm_feats,hmm_weights,alltags)[0],'hmm')
print scorer.accuracy(confusion)

0.745386688783
